In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE104954'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tubulointerstitial transcriptome from ERCB subjects with chronic kidney disease and living donor biopsies."
!Series_summary	"summary : Tubulointerstitial transcriptome from ERCB subjects with chronic kidney disease and living donor biopsies. Samples included in this analysis have been previously analyzed using older CDF definitions and are included under previous GEO submissions - GSE47184 (chronic kidney disease samples), and GSE32591 (IgA nephropathy samples).  "
!Series_overall_design	"RNA from the tubulointerstitial compartment of was extracted and processed for hybridization on Affymetrix microarrays, annotated using Human Entrez Gene ID custom CDF version 19."
Sample Characteristics Dictionary:
{0: ['tissue: Tubulointerstitium from kidney biopsy'], 1: ['diagnosis: Diabetic nephropathy', 'diagnosis: Focal segmental glomerulosclerosis and minimal change disease', 'diagnosis: Focal segmental glomerulosclerosis', nan, 'diagnosis: Minimal change 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np  # Import needed to fix 'nan' error

# 1. Gene Expression Data Availability
if "Affymetrix microarrays" in "!Series_overall_design\t" or "!Series_summary\t":
    is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# Extract available variables from the provided sample characteristics dictionary
sample_characteristics = {
    0: ['tissue: Tubulointerstitium from kidney biopsy'], 
    1: ['diagnosis: Diabetic nephropathy', 
        'diagnosis: Minimal change disease', 
        'diagnosis: Thin membrande disease', 
        'diagnosis: Hypertensive nephropathy', 
        'diagnosis: Tumor nephrectomy', 
        'diagnosis: IgA nephropathy', 
        'diagnosis: Systemic lupus erythematosus', 
        'diagnosis: Membranous glomerulonephropathy', 
        'diagnosis: Focal segmental glomerulosclerosis', 
        np.nan  # Updated the 'nan' to 'np.nan'
    ]
}

# Placeholder row indices (these are guessed based on typical dataset observations)
row_index_guess = {'diagnosis': 1}  

# 2.1 Data Availability
exact_keys = {'Chronic_kidney_disease': 'diagnosis'}
for variable, field in exact_keys.items():
    if row_index_guess.get(field) is not None:
        trait_row = row_index_guess[field]  # Update this if an exact match is found.

# Assuming no keys for age and gender available in given data
age_row = gender_row = None

# 2.3 Data Type Conversion
def convert_trait(value):
    try:
        return 1 if 'diagnosis' in value and 'nephropathy' in value else 0
    except:
        return None

def convert_age(value):
    return None  # Placeholder return, need actual conversion logic if age data available later

def convert_gender(value):
    return None  # Placeholder return, need actual conversion logic if gender data available later

# 3. Save Metadata
save_cohort_info('GSE104954', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE104954.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2811029': [1], 'GSM2811030': [1], 'GSM2811031': [1], 'GSM2811032': [1], 'GSM2811033': [1], 'GSM2811034': [1], 'GSM2811035': [1], 'GSM2811036': [0], 'GSM2811037': [0], 'GSM2811038': [0], 'GSM2811039': [0], 'GSM2811040': [0], 'GSM2811041': [0], 'GSM2811042': [0], 'GSM2811043': [None], 'GSM2811044': [None], 'GSM2811045': [None], 'GSM2811046': [None], 'GSM2811047': [None], 'GSM2811048': [None], 'GSM2811049': [None], 'GSM2811050': [None], 'GSM2811051': [None], 'GSM2811052': [None], 'GSM2811053': [None], 'GSM2811054': [None], 'GSM2811055': [None], 'GSM2811056': [None], 'GSM2811057': [None], 'GSM2811058': [None], 'GSM2811059': [None], 'GSM2811060': [None], 'GSM2811061': [0], 'GSM2811062': [0], 'GSM2811063': [0], 'GSM2811064': [0], 'GSM2811065': [0], 'GSM2811066': [0], 'GSM2811067': [0], 'GSM2811068': [0], 'GSM2811069': [0], 'GSM2811070': [0], 'GSM2811071': [0], 'GSM2811072': [0], 'GSM2811073': [0], 'GSM2811074': [0], 'GSM2811075': [0], 'GSM2811076': [0], 'GSM2811077': [0], 'GSM2811078': 

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['10000_at', '10001_at', '10002_at', '10003_at', '100048912_at',
       '10004_at', '10005_at', '10006_at', '10007_at', '100093698_at',
       '10009_at', '1000_at', '10010_at', '100126791_at', '100128124_at',
       '100128640_at', '100129128_at', '100129250_at', '100129271_at',
       '100129361_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1000_at', '10000_at', '100009676_at', '10001_at', '10004_at'], 'Symbol': ['CDH2', 'AKT3', 'ZBTB11-AS1', 'MED6', 'NAALADL1'], 'SPOT_ID': ['cadherin 2', 'AKT serine/threonine kinase 3', 'ZBTB11 antisense RNA 1', 'mediator complex subunit 6', 'N-acetylated alpha-linked acidic dipeptidase-like 1'], 'ENTREZ_GENE_ID': ['1000', '10000', '100009676', '10001', '10004']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys storing probe IDs and gene symbols from the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)
